<a href="https://colab.research.google.com/github/Yashsolanki0112/ML-projects/blob/main/deep_learning__pepper_bell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New section

In [ ]:
dataset_folder = "/content/drive/MyDrive/pepper_bell_data"


In [ ]:
import os

# List contents of the dataset folder
os.listdir(dataset_folder)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']

In [ ]:
from sklearn.model_selection import train_test_split
# Function to load images and labels from the dataset folder
def load_dataset(folder_path):
    image_paths = []
    labels = []

    # Iterate over each class folder
    for class_folder in os.listdir(folder_path):
        class_folder_path = os.path.join(folder_path, class_folder)
        if os.path.isdir(class_folder_path):
            # Iterate over each image in the class folder
            for image_file in os.listdir(class_folder_path):
                image_path = os.path.join(class_folder_path, image_file)
                image_paths.append(image_path)
                labels.append(class_folder)  # Assuming folder name represents class name

    return image_paths, labels

# Load dataset
image_paths, labels = load_dataset(dataset_folder)

# Split dataset into training and testing sets
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

# Print some information about the dataset
print("Total number of images:", len(image_paths))
print("Number of training images:", len(train_image_paths))
print("Number of testing images:", len(test_image_paths))

# Now you have your training and testing data ready for further processing


Total number of images: 2475
Number of training images: 1980
Number of testing images: 495


In [ ]:
from PIL import Image

# Open an image
image = Image.open("/content/drive/MyDrive/pepper_bell_data/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG")

# Get dimensions
width, height = image.size

# Get number of channels
num_channels = len(image.getbands())

print("Image dimensions:", width, "x", height)
print("Number of channels:", num_channels)



Image dimensions: 256 x 256
Number of channels: 3


In [ ]:

# Get list of class names (folder names)
class_names = os.listdir(dataset_folder)

# Count unique classes
num_classes = len(class_names)

print("Number of unique classes:", num_classes)


Number of unique classes: 2


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (256, 256, 3)  # Update with your image dimensions
num_classes = 2  # Update with the number of unique classes in your dataset

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 230400)            0

In [ ]:
import pandas as pd

# Create DataFrames for training and testing data
train_df = pd.DataFrame({'image_path': train_image_paths, 'label': train_labels})
test_df = pd.DataFrame({'image_path': test_image_paths, 'label': test_labels})


In [ ]:
# Define batch size and number of epochs
batch_size = 32
epochs = 5

# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Define the ImageDataGenerator with rescaling
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

# Create data generators for training and testing
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,  # DataFrame containing paths and labels for training images
    x_col="image_path",  # Column name containing image paths
    y_col="label",       # Column name containing labels
    target_size=(256, 256),  # Update with your desired image dimensions
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,    # DataFrame containing paths and labels for testing images
    x_col="image_path",   # Column name containing image paths
    y_col="label",        # Column name containing labels
    target_size=(256, 256),  # Update with your desired image dimensions
    batch_size=batch_size,
    class_mode='binary'
)


# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


Found 1980 validated image filenames belonging to 2 classes.
Found 495 validated image filenames belonging to 2 classes.
Epoch 1/5
61/61 [==============================] - 281s 5s/step - loss: 0.1552 - accuracy: 0.9476 - val_loss: 0.1197 - val_accuracy: 0.9583
Epoch 2/5
61/61 [==============================] - 281s 5s/step - loss: 0.0908 - accuracy: 0.9743 - val_loss: 0.1075 - val_accuracy: 0.9688
Epoch 3/5
61/61 [==============================] - 280s 5s/step - loss: 0.0702 - accuracy: 0.9764 - val_loss: 0.1237 - val_accuracy: 0.9583
Epoch 4/5
61/61 [==============================] - 278s 5s/step - loss: 0.0401 - accuracy: 0.9877 - val_loss: 0.3471 - val_accuracy: 0.8979
Epoch 5/5
61/61 [==============================] - 281s 5s/step - loss: 0.1807 - accuracy: 0.9389 - val_loss: 0.1890 - val_accuracy: 0.9312


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


16/16 [==============================] - 20s 1s/step - loss: 0.1841 - accuracy: 0.9333
Test Loss: 0.18410423398017883
Test Accuracy: 0.9333333373069763


In [ ]:
# Make predictions
predictions = model.predict(test_generator)


16/16 [==============================] - 21s 1s/step


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert predictions to binary values
predicted_classes = (predictions > 0.5).astype(int)

# Print classification report
#print(classification_report(test_generator.classes, predicted_classes))

# Print confusion matrix
#print(confusion_matrix(test_generator.classes, predicted_classes))


In [ ]:
# Save the model
model.save("my_model.h5")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model("my_model.h5")  # Update with the path to your saved model

# Load the sample image
img_path = "/content/0a64655c-4052-4e5f-a696-2a6cf25d10c9___GCREC_Bact.Sp 6017.JPG"  # Update with the path to your sample image
img = image.load_img(img_path, target_size=(256, 256))  # Update target_size according to your model

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the pixel values

# Make predictions
predictions = model.predict(img_array)

# Interpret the results
predicted_class = "infected" if predictions[0][0] > 0.7 else "healthy"
print("The image contains", predicted_class, "pepper bell leaves.")


1/1 [==============================] - 0s 125ms/step
The image contains infected pepper bell leaves.
